**Map visualisation**

Prepare data of sentiments as a [GeoJSON](https://geojson.org/) FeatureCollection for using on a OpenStreetMap with the [vanilla-js-web-component-leaflet-geojson ](https://github.com/migupl/vanilla-js-web-component-leaflet-geojson) to look like [this](https://migupl.github.io/vanilla-js-web-component-leaflet-geojson/example.html).

**Setup**

In [ ]:
!pip install pandas

**Preparando los datos**

In [2]:
import pandas as pd

versos_al_paso_geo_url = 'https://github.com/migupl/sentimientos-al-paso/raw/main/notebooks/output/versosalpaso_sentiment_text-davinci-003_geo.csv'
versos_al_paso_geo = pd.read_csv(versos_al_paso_geo_url, sep=';', encoding='utf-8')
versos_al_paso_geo.columns

Index(['Unnamed: 0', 'id', 'latitud', 'longitud', 'autor', 'barrio', 'verso',
       'direccion', 'sentiment', 'quarter', 'district', 'city'],
      dtype='object')

In [3]:
sentiments_by_district = versos_al_paso_geo.groupby('district')
pd.pivot_table(versos_al_paso_geo[['district', 'sentiment']], index='district', columns='sentiment', aggfunc=len, fill_value=0)
sentiments_by_district

In [4]:
no_of_districts = len(versos_al_paso_geo.district.unique())
print(f'There are {no_of_districts} districts')

There are 21 districts


In [5]:
pd.pivot_table(versos_al_paso_geo[['district', 'sentiment']], index='district', columns='sentiment', aggfunc=len, fill_value=0)

sentiment,negative,neutral,positive
district,,,
Arganzuela,1,4,48
Barajas,0,0,11
Carabanchel,2,7,72
Centro,1,2,67
Chamartín,0,4,71
Chamberí,1,0,63
Ciudad Lineal,0,1,77
Fuencarral-El Pardo,2,4,59
Hortaleza,0,5,51


**Define GeoJSON Feature**

In [6]:
import json

def geojson_feature(latitude: float, longitude: float, author: str, verse: str, sentiment: str) -> json:
    colors_name = {'negative': 'red', 'neutral': 'yellow', 'positive': 'green'} 
    return {
        'type': "Feature",
        'geometry': {
            'type': "Point",
            'coordinates': [longitude, latitude]
        },
        'properties': {
            'popupContent': f'<strong>{verse}</strong><br>— <cite>{author}</cite>',
            'icon': {
                "iconUrl": f'https://raw.githubusercontent.com/migupl/svg-vectors-and-icons/main/heart-like/heart-{colors_name[sentiment]}.png',
                "iconSize": [41, 41],
                "iconAnchor": [20, 41],
                "popupAnchor": [1, -34]
            },
        }
    }

Prepare the GeoJSON's FeatureCollection by district

In [7]:
geojson = {}
for name, group in sentiments_by_district:
    features = []
    for index, row in group.iterrows():
        feature = geojson_feature(row.latitud, row.longitud, row.autor, row.verso, row.sentiment)
        features.append(feature)

    geojson[name] = {
        'type': "FeatureCollection",
        'features': features
    }
    
keys = geojson.keys()
keys

dict_keys(['Arganzuela', 'Barajas', 'Carabanchel', 'Centro', 'Chamartín', 'Chamberí', 'Ciudad Lineal', 'Fuencarral-El Pardo', 'Hortaleza', 'Latina', 'Moncloa-Aravaca', 'Moratalaz', 'Puente de Vallecas', 'Retiro', 'Salamanca', 'San Blas - Canillejas', 'Tetuán', 'Usera', 'Vicálvaro', 'Villa de Vallecas', 'Villaverde'])

Test that there are 21 districts

In [8]:
import unittest

tc = unittest.TestCase()
no_of_keys = len(keys)
tc.assertEqual(21, no_of_keys)

Save JSON for later use

In [12]:
js_content = f"""
const data = {geojson};
export {{ data }}
"""

In [14]:
js_file_path = './output/sentiments_by_district_geo.js'
with open(js_file_path, 'w') as f:
    f.write(js_content)